# Chi Squared - Commented and smoothed out

## Import modules

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sncosmo
from bayesn import SEDmodel
import matplotlib.patches as mpatches
import os
import pandas as pd
import pickle

## Make a list of SNe

In [2]:
directory = os.fsencode("/root/partiiiproject/Archive")
supernovae_names = ["sn2006hb"]
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".dat"):
        supernovae_names.append(filename[:-4])
        continue
    else:
        continue

## Decide on Dust Laws

In [3]:
dust_laws = ('F99', 'F19', 'G23')

## Load SED model

In [4]:
filter_yaml = "/root/partiiiproject/bayesn-filters/bayesn-filters/filters.yaml"
model = SEDmodel(load_model='W22_model', filter_yaml = filter_yaml)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Current devices: [CpuDevice(id=0), CpuDevice(id=1), CpuDevice(id=2), CpuDevice(id=3)]
Currently working in /root/partiiiproject
Loading built-in model W22_model
Loading built-in reddening law F99


## Open existing data frame / or make a new one

In [5]:
#Decide on method
method = "sim" #"fit" or "sim"

#Decide new or existing?
new_df = False

#df directory name
df_directory = "ChiSquaredFinal"+method+".csv"

if new_df:
    #New data frame
    df = pd.DataFrame(columns=dust_laws)
    df = pd.DataFrame(df, index=supernovae_names)
    df.to_csv(df_directory)
    print(df)
else:
    #Open existing data frame
    df = pd.read_csv(df_directory)
    df = df.drop(['Unnamed: 0'], axis=1)
    df.index = supernovae_names
    print(df)

                 F99         F19         G23
sn2006hb   90.263402         NaN  329.111540
sn2006hb   90.263402         NaN  329.111540
sn2008fl   25.140620  444.505339   33.405831
sn2007jg   76.807409  103.665883   87.763588
sn2008R    26.738986  605.211319   29.863422
...              ...         ...         ...
sn2007hj   10.944924   14.408217  169.981562
sn2005ki  132.027520  221.541212   20.304248
sn2007bc   18.830460    9.771580  347.240464
sn2005iq    3.791245   53.359055  105.925775
sn2008bq   48.185840  245.957264  170.495173

[91 rows x 3 columns]


## Get Chi Squared values for a set of SNe

In [8]:
progress = 0 # A counter for hwo many we have done
for supernova_name in supernovae_names[77:]:

    #Get SN info from the original data file
    supernova_file = "/root/partiiiproject/Archive/"+supernova_name+".dat"
    meta, lcdata = sncosmo.read_snana_ascii(supernova_file, default_tablename='OBS')
    lcdata = lcdata['OBS'].to_pandas()

    #Extract useful properties from data file used in fits and simulation
    bands = np.unique(lcdata['FLT'])
    z = meta['REDSHIFT_FINAL']
    ebv_mw = meta['MWEBV']
    
    #Get the observed times covered by the model, so in range [-10,40]
    t_obs = lcdata['MJD'] - meta['SEARCH_PEAKMJD']
    t_obs = t_obs[(-10<=t_obs) & (t_obs<=40)]
    t_obs = np.unique(t_obs)

    #Repeat through each dust law for that SN
    for dust_law in dust_laws:

        #Define directory where we should have our fits saved
        try:
            sampleChainsDir = "DustLawsFitOfAllSamples/"+supernova_name + dust_law + "_chains.pkl"
            sampleSummaryDir = "DustLawsFitOfAllSamples/"+supernova_name + dust_law + "_fit_summary.csv"

        except FileNotFoundError:
            continue
        #Check we have generated fits for this SN, otherwise move onto next SN
        try:
            #Load the chains with pickle
            with open(sampleChainsDir, 'rb') as file:
                chains = pickle.load(file)
        except FileNotFoundError:
            continue

        if method == "sim":
            #Number of simulations to load
            N=1
            #Get values for simulation directly from chains, taking the mean
            fit_mu = np.mean(chains['mu'])
            fit_theta = np.mean(chains['theta'])
            fit_RV = np.mean(chains['RV'])
            fit_AV = np.mean(chains['AV'])
            fit_tmax = np.mean(chains['tmax'])
            fit_delM = np.mean(chains['delM'])
            
            #For epsilon we need some reshaping to get it in the correct form
            fit_eps_inner = chains['eps']
            fit_eps_inner = np.reshape(fit_eps_inner, (4, 250, 9, 6), order='F')
            fit_eps = np.zeros((4, 250, 11, 6))
            fit_eps[:,:,1:-1,:] = fit_eps_inner
            fit_eps = fit_eps.mean(axis = 1)
            fit_eps = fit_eps.mean(axis = 0)
            fit_eps = np.reshape(fit_eps, (1 , 11, 6))

            #Run simulation and get flux values
            try:
                sim = model.simulate_light_curve(t_obs, N, bands, mu=fit_mu, theta=fit_theta,  z=z, ebv_mw=ebv_mw, mag=False, write_to_files=False, AV=fit_AV, RV=fit_RV, redlaw=dust_law, tmax=fit_tmax, eps=fit_eps)
            except ValueError:
                continue
            flux, flux_err, params = sim
            flux = np.reshape(flux,(len(bands),len(t_obs)))

        if method == "fit":
            #Use get flux from chains to get the best fit
            try:
                flux_grid = model.get_flux_from_chains(t_obs, bands, sampleChainsDir, z, ebv_mw, mag=False, num_samples=10)
                flux, flux_err = flux_grid.mean(axis=(0, 1)), flux_grid.std(axis=(0, 1))
            except ValueError:
                continue
        
        chi_squared = np.zeros(len(lcdata))
        for i, entry in lcdata.iterrows():
            #Remove the U bands
            if entry.FLT in ['u_CSP', 'U']:
                continue
            index_t_obs = np.where(t_obs == (entry.MJD-meta['SEARCH_PEAKMJD']))
            index_band = np.where(bands == entry.FLT)
            Exp = flux[index_band, index_t_obs]
            Obs = entry.FLUXCAL
            ObsErr = entry.FLUXCALERR
            chi_squared_calc = ((Obs-Exp)**2)/(ObsErr**2)
            if chi_squared_calc.size != 0:
                chi_squared[i] = chi_squared_calc
                
        #Remove all zero values before taking the mean
        chi_squared = chi_squared[chi_squared>0]
        df.at[supernova_name, dust_law] = np.mean(chi_squared)
        #Record progress for my convenience
        progress = progress + 1
        print(str(progress)+ ' out of '+str(len(supernovae_names)*len(dust_laws)))
        #Save after each SN is calculated
        df.to_csv(df_directory)


/tmp/ipykernel_7801/2502307303.py:85: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  chi_squared[i] = chi_squared_calc


1 out of 273


/tmp/ipykernel_7801/2502307303.py:85: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  chi_squared[i] = chi_squared_calc


2 out of 273


/tmp/ipykernel_7801/2502307303.py:85: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  chi_squared[i] = chi_squared_calc


...

/tmp/ipykernel_7801/2502307303.py:85: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  chi_squared[i] = chi_squared_calc


40 out of 273


/tmp/ipykernel_7801/2502307303.py:85: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  chi_squared[i] = chi_squared_calc


41 out of 273
42 out of 273


/tmp/ipykernel_7801/2502307303.py:85: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  chi_squared[i] = chi_squared_calc


## Print the dataframe to see what we have managed

In [ ]:
print(df)